# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

The first step is to randomly assign points to a cluster. Then, you calculate the mean (centroid) of all points is assigned to that cluster. Next, the points are reassigned to the cluster whose centroid is closest. This is repeated until the cluster assignments stop changing.

#### b) Create the builder function for step 1.

In [24]:
random_label <- function(points_n, clusters_n) {
  sample(rep(1:clusters_n, length.out=points_n))
}

#### c) Create the builder function for step 2.

In [25]:
assign_cluster <- function(df, means) {
  labels <- c()
  for (i in 1:nrow(df)) {
    point <- df[i, ]
    dists <- apply(means[, -1, drop=FALSE], 1, function(center){
      sum((as.numeric(point) - center)^2)
    })
    labels <- c(labels, means$cluster[which.min(dists)])
  }
  labels
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [26]:
get_cluster_means <- function(df, labels) {
  df |>
    mutate(cluster = labels) |>
    group_by(cluster)|>
    summarize(across(everything(), mean), .groups="drop") |>
    arrange(cluster)
}

#### e) Create the builder function for step 4.

In [27]:
kmeans_end <- function(old_means, new_means, eps=1e-6) {
  om <- as.matrix(old_means[, -1, drop=FALSE])
  nm <- as.matrix(new_means[, -1, drop=FALSE])
  diff <- sqrt(rowSums((om-nm)^2))
  mean(diff) < eps
}

#### f) Combine them all into your own `kmeans` function.

In [28]:
my_kmeans <- function(df, clusters_n, eps=1e-6, max_i = 100) {
  labels <- random_label(nrow(df), clusters_n)
  old_means <- get_cluster_means(df, labels)
  done <- FALSE
  i <- 0

  while(!done && i<max_i) {
    i <- i+1
    labels <- assign_cluster(df, old_means)
    new_means <- get_cluster_means(df, labels)
    if (kmeans_end(old_means,new_means,eps)){
      done <- TRUE
    }
    old_means <- new_means
  }
  list(labels = labels, means = new_means, iter=i)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [29]:
library(tidyverse)
voltages_df <- read_csv("voltages_df.csv")

Rows: 900 Columns: 250
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (250): 0, 1.00401606425703, 2.00803212851406, 3.01204819277108, 4.016064...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [30]:
results <- my_kmeans(voltages_df,3)
print(results$labels)
print(results$means)

  [1] 2 3 3 3 3 3 1 1 3 2 2 2 1 2 3 2 2 2 2 2 3 3 3 3 3 2 2 2 1 1 1 1 3 1 1 3 3
 [38] 2 1 3 2 2 2 1 3 1 1 3 1 1 3 1 2 2 1 2 3 2 1 3 2 2 3 2 1 1 3 1 2 1 2 3 1 1
 [75] 3 3 2 1 1 1 1 1 3 1 1 3 3 3 2 1 1 1 3 1 3 3 1 3 3 1 1 1 3 1 2 2 1 2 2 3 2
[112] 2 3 1 2 1 2 2 3 3 3 3 2 2 1 3 3 1 2 1 3 2 3 1 2 3 3 3 2 1 2 3 3 1 2 2 3 2
[149] 2 1 2 1 3 3 1 2 1 1 3 2 1 1 2 3 2 1 3 2 3 2 2 1 3 2 1 1 3 1 2 3 1 3 1 2 1
[186] 3 3 2 3 2 2 3 2 2 1 1 2 3 2 2 1 2 1 1 3 2 3 1 3 2 3 1 2 1 1 3 1 1 3 3 3 3
[223] 3 3 3 1 1 1 1 2 3 2 1 3 3 1 1 2 2 1 1 2 1 1 3 1 2 1 1 3 1 3 2 1 3 1 1 2 2
[260] 2 3 3 1 2 3 3 3 1 1 1 2 3 3 3 1 2 1 1 3 3 1 2 1 2 2 3 1 3 3 1 1 2 1 1 2 3
[297] 2 1 1 3 3 1 2 2 2 1 2 3 3 3 2 2 2 2 2 2 3 2 3 2 2 1 3 1 2 2 2 2 3 2 3 2 1
[334] 2 3 3 1 1 2 3 1 3 1 1 2 1 3 1 2 2 1 3 3 3 2 1 3 2 1 1 3 3 3 3 2 1 2 3 2 2
[371] 2 2 2 3 3 1 1 2 2 2 2 1 1 2 2 1 1 3 3 1 2 3 3 3 3 1 3 3 1 2 2 2 2 3 3 2 2
[408] 1 1 1 3 1 1 2 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 3 1 1 3 1 2 3 2 2 2 1 1 1 1 2
[445] 3 2 2 2 1 3 2 2 2 3 3 2 3 3 2 1 3 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [31]:
results_r <- kmeans(as.matrix(voltages_df), centers = 3)
print(results_r$cluster)
print(results_r$centers)

  [1] 3 1 1 1 1 1 2 2 1 3 3 3 2 3 1 3 3 3 3 3 1 1 1 1 1 3 3 3 2 2 2 2 1 2 2 1 1
 [38] 3 2 1 3 3 3 2 1 2 2 1 2 2 1 2 3 3 2 3 1 3 2 1 3 3 1 3 2 2 1 2 3 2 3 1 2 2
 [75] 1 1 3 2 2 2 2 2 1 2 2 1 1 1 3 2 2 2 1 2 1 1 2 1 1 2 2 2 1 2 3 3 2 3 3 1 3
[112] 3 1 2 3 2 3 3 1 1 1 1 3 3 2 1 1 2 3 2 1 3 1 2 3 1 1 1 3 2 3 1 1 2 3 3 1 3
[149] 3 2 3 2 1 1 2 3 2 2 1 3 2 2 3 1 3 2 1 3 1 3 3 2 1 3 2 2 1 2 3 1 2 1 2 3 2
[186] 1 1 3 1 3 3 1 3 3 2 2 3 1 3 3 2 3 2 2 1 3 1 2 1 3 1 2 3 2 2 1 2 2 1 1 1 1
[223] 1 1 1 2 2 2 2 3 1 3 2 1 1 2 2 3 3 2 2 3 2 2 1 2 3 2 2 1 2 1 3 2 1 2 2 3 3
[260] 3 1 1 2 3 1 1 1 2 2 2 3 1 1 1 2 3 2 2 1 1 2 3 2 3 3 1 2 1 1 2 2 3 2 2 3 1
[297] 3 2 2 1 1 2 3 3 3 2 3 1 1 1 3 3 3 3 3 3 1 3 1 3 3 2 1 2 3 3 3 3 1 3 1 3 2
[334] 3 1 1 2 2 3 1 2 1 2 2 3 2 1 2 3 3 2 1 1 1 3 2 1 3 2 2 1 1 1 1 3 2 3 1 3 3
[371] 3 3 3 1 1 2 2 3 3 3 3 2 2 3 3 2 2 1 1 2 3 1 1 1 1 2 1 1 2 3 3 3 3 1 1 3 3
[408] 2 2 2 1 2 2 3 1 1 1 2 1 1 1 3 3 1 3 2 1 1 1 1 2 2 1 2 3 1 3 3 3 2 2 2 2 3
[445] 1 3 3 3 2 1 3 3 3 1 1 3 1 1 3 2 1 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

They are not the same since the centroids in the first step lead to random initial centroids so the cluster numbering and assignment can shift every time the code is run. Thus, the clusters/labels might not match. The cluster means are similar to one another but they are not identical; however, the clusters would showcase similar patterns attributing to the similar cluster means.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

You can loop through each point and then compute its distance to all centroids and then assign it to the closest one. This is more intuitive; however, this is can take a very long time to carry out.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

Instead of use a for loop to assign the clusters for kmeans we can use vectorized distance computations which allows R to perfom element-wise operations across the dataset in one step making the process much faster than the initial for loop.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectorizing is more efficient because it runs much faster for large dataset since a loop is having to be evaluated for every single data point.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

K-means fails when the clusters are not spherical in shape, not of similar size, or not well separated (i.e. are overlapping). K-means assumes that clusters are roughly spherical Gaussians of similar size and spread, and that the distances can be measured in a vector space. When these assumptions do not hold, the algorithm may misclassify points or lead to convergance.

## Question 5
#### What assumption do Guassian mixture models make?

Gaussian Mixture Models (GMMs) assume that data are generated from a mixture of Gaussian distributions, each with its own mean and covariance matrix. Unlike K-means, GMMs allow for each point to belong to clusters with certain probabilities.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering assumes that data can be represented as a graph, where edges connect similar points. It uses the eigenvectors of the Laplacian graph to find clusters. This helps because it does not assume spherical or equal-sized clusters. So, it can identify arbitrarily shaped clusters and detect structure based on connectivity rather than the Euclidean distance alone.

## Question 7
#### Define the gap statistic method. What do we use it for?

The gap statistic compares the total within-cluster variation for different values of k (number of clusters) to the expected under a null reference distribution. It is used to determine the optimal number of clusters, which is the value of k where the gap statistic is maximized. This indicates that the clustering is significantly better than random uniform clustering.